In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ratings_df = pd.read_csv("/content/drive/MyDrive/recomendacao/ratings_small.csv")

In [ ]:
ratings_df.movieId.nunique()

9724

In [ ]:
movies_df = pd.read_csv("/content/drive/MyDrive/recomendacao/movies_small.csv")

In [ ]:
movies_df.shape

(9742, 3)

In [ ]:
df = pd.merge(movies_df, ratings_df, on = "movieId")

In [ ]:
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
combine_movie_rating = df.dropna(axis =0, subset = ['title'])
movie_ratingCount = (combine_movie_rating.
     groupby(by=["title"])["rating"].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
      [['title', 'totalRatingCount']]
      )

In [ ]:
movie_ratingCount.sort_values(by="totalRatingCount", ascending = False)

,title,totalRatingCount
3158,Forrest Gump (1994),329
7593,"Shawshank Redemption, The (1994)",317
6865,Pulp Fiction (1994),307
7680,"Silence of the Lambs, The (1991)",279
5512,"Matrix, The (1999)",278
...,...,...
4773,King Solomon's Mines (1950),1
4772,King Solomon's Mines (1937),1
4771,King Ralph (1991),1
4769,King Kong Lives (1986),1


In [ ]:
df = pd.merge(df, movie_ratingCount, on = "title")

In [ ]:
df.head()

,movieId,title,genres,userId,rating,timestamp,totalRatingCount
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,215
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,215
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946,215
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970,215
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483,215


In [ ]:
movie_ratingCount.query("totalRatingCount <= 3").shape

(5539, 2)

## Pivotando os Dados

In [ ]:
user_ratings_table = df.pivot_table(index="userId",columns="title", values="rating")

In [ ]:
user_ratings_table

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.5,3.5,0.0,0.0,0.0


In [ ]:
user_ratings_table.fillna(0,inplace=True)

In [ ]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(user_ratings_table.values)

O código importa a função csr_matrix do módulo scipy.sparse e então cria uma representação de matriz esparsa dos dados contidos no DataFrame user_ratings_table usando a função csr_matrix. A csr_matrix, ou matriz de linha esparsa comprimida, é uma representação de matriz esparsa que é eficiente para computação e uso de memória quando a maioria dos elementos na matriz é zero. Este é frequentemente o caso em sistemas de recomendação onde apenas uma pequena porcentagem de itens é avaliada por um usuário.

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
movie_knn = NearestNeighbors(metric ='cosine', algorithm ='brute')

In [ ]:
movie_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
query_index = np.random.choice(user_ratings_table.shape[0])

In [ ]:
query_index

393

In [ ]:
distances, indices = movie_knn.kneighbors(user_ratings_table.iloc[query_index,:].values.reshape(1,-1), n_neighbors=6)

In [ ]:
lista_user = []
for i in range(0, len(distances.flatten())):
  if i == 0:
    print("Recommendations for {0}:\n".format(user_ratings_table.index[query_index]))
  else:
    print('{0}: {1}, com distância de {2}'.format(i,user_ratings_table.index[indices.flatten()[i]], distances.flatten()[i]))
    lista_user.append(user_ratings_table.index[indices.flatten()[i]])

Recommendations for 394:

1: 379, com distância de 0.38443829316349665
2: 126, com distância de 0.3973252068823856
3: 507, com distância de 0.4580162232793845
4: 46, com distância de 0.46033878337031775
5: 498, com distância de 0.4895395677026705


In [ ]:
lista_user

[379, 126, 507, 46, 498]

In [ ]:
userid_list = lista_user

In [ ]:
filmes_assistidos = df[df.userId == query_index].title.unique()

In [ ]:
filmes_assistidos[:10]

array(['Braveheart (1995)',
       'Léon: The Professional (a.k.a. The Professional) (Léon) (1994)',
       'Fugitive, The (1993)', 'Blade Runner (1982)',
       'Terminator 2: Judgment Day (1991)',
       'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
       'Trainspotting (1996)', 'Reservoir Dogs (1992)',
       'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
       'Brazil (1985)'], dtype=object)

In [ ]:
recomendacoes = df[df.userId.isin(userid_list)].drop_duplicates(subset="movieId")

In [ ]:
recomendacoes.head()

,movieId,title,genres,userId,rating,timestamp,totalRatingCount
16,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,46,5.0,834787906,215
577,7,Sabrina (1995),Comedy|Romance,498,4.0,839197378,54
623,10,GoldenEye (1995),Action|Adventure|Thriller,46,3.0,834787826,132
1096,19,Ace Ventura: When Nature Calls (1995),Comedy,379,2.0,847397556,88
1200,21,Get Shorty (1995),Comedy|Crime|Thriller,379,2.0,847397556,89


In [ ]:
recomendacoes = recomendacoes[~recomendacoes.isin(filmes_assistidos)]

In [ ]:
recomendacoes.query('totalRatingCount > 100')

,movieId,title,genres,userId,rating,timestamp,totalRatingCount
16,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,46,5.0,834787906,215
623,10,GoldenEye (1995),Action|Adventure|Thriller,46,3.0,834787826,132
1515,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,46,4.0,834788094,177
1704,34,Babe (1995),Children|Drama,126,3.0,845925854,128
1887,39,Clueless (1995),Comedy|Romance,46,3.0,834788093,104
2151,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,126,5.0,845925822,203
2399,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,46,5.0,834788094,204
3404,110,NaN,Action|Drama|War,46,5.0,834788016,237
3629,111,Taxi Driver (1976),Crime|Drama|Thriller,46,4.0,834788094,104
4097,150,Apollo 13 (1995),Adventure|Drama|IMAX,126,4.0,845925577,201


In [ ]:
df[df.userId == query_index].title.unique

<bound method Series.unique of 3536                                     Braveheart (1995)
7811     Léon: The Professional (a.k.a. The Professiona...
12722                                 Fugitive, The (1993)
14693                                  Blade Runner (1982)
15798                    Terminator 2: Judgment Day (1991)
                               ...                        
96977                    X-Men: Days of Future Past (2014)
97381                       Guardians of the Galaxy (2014)
98208                       Avengers: Age of Ultron (2015)
98657                     The Amazing Screw-On Head (2006)
98680                                 Power/Rangers (2015)
Name: title, Length: 123, dtype: object>